### **Libraries**

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import os

### **Set working directory to root**

In [3]:
%pwd
os.chdir("../")
%pwd

'd:\\MLops\\NetworkSecurity'

### **Load the data**

In [4]:
pd.set_option("display.max_columns", None)

CLEAN_DATA = Path("data/processed/phiusiil_clean_deduped.csv")
OUTPUT_DIR = Path("data/processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Loading deduplicated dataset from EDA")
df = pd.read_csv(CLEAN_DATA)

print(f"\nDataset shape: {df.shape}")
print(f"Unique URLs: {df['URL'].nunique():,}")
print(f"Class balance:")
print(df["label"].value_counts())


Loading deduplicated dataset from EDA

Dataset shape: (235370, 56)
Unique URLs: 235,370
Class balance:
label
1    134850
0    100520
Name: count, dtype: int64


In [ ]:
print(f"Total unique domains: {df['Domain'].nunique()}")
print(f"Total URLs: {len(df)}")
print(f"Average URLs per domain: {len(df) / df['Domain'].nunique():.2f}")

# Check if domains repeat across BOTH classes
domain_class_dist = df.groupby("Domain")["label"].apply(lambda x: x.nunique())
mixed_label_domains = (domain_class_dist > 1).sum()
print(f"Domains with BOTH phish and legit examples: {mixed_label_domains}")


Total unique domains: 220086
Total URLs: 235370
Average URLs per domain: 1.07
Domains with BOTH phish and legit examples: 54


: 